In [1]:
# Import libraries

from bs4 import BeautifulSoup
import requests
import smtplib
import time
import datetime
import csv
import datetime
import pandas as pd
import threading


In [2]:
# Connect to website
URL = 'https://www.amazon.nl/Tamisom-Halterrek-Gewichtsrek-Dumbbell-Storage/dp/B0B5MSGCP9/ref=sr_1_3_mod_primary_new?crid=3V4SAO5OV5OSF&keywords=dumbbells+rek+alleen&qid=1701077744&refinements=p_72%3A4993218031&rnid=4993217031&sbo=RZvfv%2F%2FHxDF%2BO5021pAnSA%3D%3D&sprefix=%2Caps%2C160&sr=8-3'

# User-Agent: https://httpbin.org/get
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36'
}

page = requests.get(URL, headers=headers)

soup1 = BeautifulSoup(page.content, 'html.parser')

soup2 = BeautifulSoup(soup1.prettify(), 'html.parser')

title = soup2.find(id='productTitle').get_text(strip=True)

price_element = soup2.find('span', class_='a-price')
price = price_element.find('span', class_='a-offscreen').get_text(strip=True)

print(title)
print(price)


Tamisom Halterrek Stand, 2 Tier Gewichtsrek voor Home Gym Dumbbell Storage, Alleen rek, 32,5 "x24,5 "X4.4"
€193,82


In [3]:
# remove the euro sign
price = price[1:]

In [4]:
today = datetime.date.today()

print(today)

2023-12-13


In [5]:
header = ['Title', 'Price', 'Date']
data = [title, price, today]

#Ran the following once, commenting them out not so that it does not reset

#with open('WebScraperDataset.csv', 'w', newline='', encoding='UTF8') as f:
    #writer = csv.writer(f)
    #writer.writerow(data)

In [6]:
df = pd.read_csv(r'C:\Users\krijn\WebScraperDataset.csv')
print(df)

Empty DataFrame
Columns: [Tamisom Halterrek Stand, 2 Tier Gewichtsrek voor Home Gym Dumbbell Storage, Alleen rek, 32,5 "x24,5 "X4.4", 193,82, 2023-12-13]
Index: []


In [7]:
# Appending data to the csv file

with open('WebScraperDataset.csv', 'a', newline='', encoding='UTF8') as f:
    writer = csv.writer(f)
    writer.writerow(data)

In [8]:
# Send me an email if price is below a set price!

def send_email():
    server = smtplib.SMTP_SSL('smtp.gmail.com',465)
    server.ehlo()
    #server.starttls()
    server.ehlo()
    server.login('mijin.gwak@outlook.com','xxxxxxxxxxxxxx')
    
    subject = "The dumbbell rack you want is below €100! Now is your chance to buy!"
    body = "Mijin, This is the moment we have been waiting for. Now is your chance to pick up the shirt of your dreams. Don't mess it up! Link here: https://www.amazon.nl/Tamisom-Halterrek-Gewichtsrek-Dumbbell-Storage/dp/B0B5MSGCP9/ref=sr_1_3_mod_primary_new?crid=3V4SAO5OV5OSF&keywords=dumbbells+rek+alleen&qid=1701077744&refinements=p_72%3A4993218031&rnid=4993217031&sbo=RZvfv%2F%2FHxDF%2BO5021pAnSA%3D%3D&sprefix=%2Caps%2C160&sr=8-3"
   
    msg = f"Subject: {subject}\n\n{body}"
    
    server.sendmail(
        'mijin.gwak@outlook.com',
        msg
     
    )

In [9]:
# Automate checking the price each day by combining all of the above code into one function

def check_price():
    URL = 'https://www.amazon.nl/Tamisom-Halterrek-Gewichtsrek-Dumbbell-Storage/dp/B0B5MSGCP9/ref=sr_1_3_mod_primary_new?crid=3V4SAO5OV5OSF&keywords=dumbbells+rek+alleen&qid=1701077744&refinements=p_72%3A4993218031&rnid=4993217031&sbo=RZvfv%2F%2FHxDF%2BO5021pAnSA%3D%3D&sprefix=%2Caps%2C160&sr=8-3'

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36'
    }

    page = requests.get(URL, headers=headers)

    soup1 = BeautifulSoup(page.content, 'html.parser')

    soup2 = BeautifulSoup(soup1.prettify(), 'html.parser')

    title = soup2.find(id='productTitle').get_text(strip=True)

    price_element = soup2.find('span', class_='a-price')
    price = price_element.find('span', class_='a-offscreen').get_text(strip=True)
    
    # \Removing the euro sign replacing , with . so that it can be converted it into a float
    price = float(price[1:].replace(',', '')) 
    today = datetime.date.today()
    
    header = ['Title', 'Price', 'Date']
    data = [title, price, today]
    
    with open('WebScraperDataset.csv', 'a', newline='', encoding='UTF8') as f:
        writer = csv.writer(f)
        writer.writerow(data)
        
    if(price < 100):
        send_email() # If it is cheaper than 100 euros then send me an email!!
    

In [ ]:
# Run until the stop_flag is True

stop_flag = False

while not stop_flag:
    check_price()
    time.sleep(86400)  # Run every 24 hours

In [ ]:
# Change the stop_flag to True so that the loop stops running
#stop_flag = True